# Analyse hydrogen market results from PyPSA-Earth network
This notebook analyze the hydrogen market data of the network (.nc) file. It generates comprehensive plots and summaries to visualize the results.
* Hydrogen Market Value
* Hydrogen Production Cost
* Hydrogen Capacity 
* Hydrogen Export 

Sources: 
- locAL DEcarbonization and HYDrogen Export: https://github.com/energyLS/aldehyde 
- Statistics module: https://pypsa.readthedocs.io/en/latest/examples/statistics.html
- Templates and colors: https://github.com/pypsa-meets-earth/documentation/blob/main/notebooks/network_analysis.ipynb

File is needed:
* PyPSA network file (.nc) includes the hydrogen financing information.

## Import packages

In [ ]:
import yaml
import pypsa
import warnings
import matplotlib.pyplot as plt
import os
import sys
import glob

## Path settings
This section reads the config parameters from your config.yaml file and automatically reads the output of the optimization with those settings

Some sample network file names:
- elec_s_200_ec_lcopt_Co2L-3h_144h_2030_0.071_AB_10export.nc
- elec_s10_10_ec_lcopt_Co2L_3H.nc
- elec_s_10_ec_lc3.0_Co2L_3H_2035_0.071_AB_14export.nc

In [ ]:
# change to parent folder
if not os.path.isdir("pypsa-earth"):
    os.chdir("../..")
PARENT = os.path.realpath("pypsa-earth") + os.sep

with open(os.path.join(PARENT, "config.yaml"), "r") as f:
    cfg = yaml.safe_load(f)

In [ ]:
run = cfg.get("run", {}) or {}
scen = cfg.get("scenario", {}) or {}
costs = cfg.get("costs", {}) or {}
expo = cfg.get("export", {}) or {}
results_dir = (cfg.get("results_dir", "results/") or "results/").strip("/") + "/"

RDIR = (run.get("name") or "").strip() + ("/" if run.get("name") else "")
SECDIR = (run.get("sector_name") or "").strip() + ("/" if run.get("sector_name") else "")

def one(val, allow_empty=False):
    """Return single concrete value; '*' if multiple/empty (unless allow_empty)."""
    if val is None: return "*"
    if isinstance(val, (list, tuple)):
        if len(val) == 1:
            s = "" if val[0] is None else str(val[0])
            return s if (allow_empty or s != "") else "*"
        return "*"
    s = "" if val is None else str(val)
    return s if (allow_empty or s != "") else "*"

simpl = one(scen.get("simpl", [""]), allow_empty=True)     # may be ''
clusters = one(scen.get("clusters"))
ll = one(scen.get("ll"))
opts = one(scen.get("opts"))
sopts = one(scen.get("sopts", [""]), allow_empty=True)     # may be ''
plan = one(scen.get("planning_horizons"))
dr = one(costs.get("discountrate"))
demand = one(scen.get("demand"))
h2_raw = one(expo.get("h2export"))
h2 = (h2_raw + "export") if h2_raw != "*" else "*export"

# directories
elec_dir = os.path.join(PARENT, results_dir, RDIR, "networks")
sector_post_dir = os.path.join(PARENT, results_dir, SECDIR, "postnetworks")
sector_pre_dir = os.path.join(PARENT, results_dir, SECDIR, "prenetworks")

# sector finals, prenetworks, electricity-only
patterns = [
    os.path.join(sector_post_dir, f"elec_s{simpl}_{clusters}_ec_l{ll}_{opts}_{sopts}_{plan}_{dr}_{demand}_{h2}.nc"),
    os.path.join(sector_pre_dir, f"elec_s{simpl}_{clusters}_ec_l{ll}_{opts}_{sopts}_{plan}_{dr}_{demand}_{h2}.nc"),
    os.path.join(sector_pre_dir, f"elec_s{simpl}_{clusters}_ec_l{ll}_{opts}_{sopts}_{plan}_{dr}_{demand}.nc"),
    os.path.join(elec_dir, f"elec_s{simpl}_{clusters}_ec_l{ll}_{opts}.nc"),
    os.path.join(elec_dir, f"elec_s{simpl}_{clusters}_ec_l{ll}_{opts}_*.nc"),  # Monte-Carlo suffix (unc)
]

# TODO: find networks from config

### Analysis Network Setup

In [ ]:
n = pypsa.Network()
statistics = n.statistics()

### Hydrogen Market

In [ ]:
# Hydrogen Market Value
h2_market = (statistics.loc["Link", "H2 Electrolysis"]["Market Value"]).round(2)
print(f"Yearly Hydrogen Market Value is: {h2_market} €")

### Hydrogen Costs

In [ ]:
h2_capex = (
        statistics.loc["Link", "H2 Electrolysis"]
        .loc["Capital Expenditure"]
        / 1e6
)  # in Mio. €
print(f"Capital Expenditure is: {h2_capex.round(2)} Mio. €")

h2_opex = (
    statistics.loc["Link", "H2 Electrolysis"]
    .loc["Operational Expenditure"]
    / 1e6
)  # in Mio. €
print(f"Operational Expenditure is: {h2_opex.round(2)} Mio. €")

hydrogen_production = (
    statistics.loc["Link", "H2 Electrolysis"]
    .loc["Supply"]
    .sum() 
    / 1e6
)  # in TWh

total_hydrogen_production_cost = h2_capex + h2_opex
hydrogen_cost_per_unit = total_hydrogen_production_cost / hydrogen_production

# Print Results
print(f"Total Hydrogen Production Cost: {total_hydrogen_production_cost.round(2)} Mio. €")
print(f"Hydrogen Production Cost per Unit: {hydrogen_cost_per_unit.round(2)} Mio. € per TWh")


### Hydrogen Exports

#### H2 Export on a hourly basis

In [ ]:
# Hourly hydrogen export prices
hydrogen_buses = n.buses[n.buses.index.str.contains("H2", case=False)]
hydrogen_prices = n.buses_t.marginal_price[hydrogen_buses.index]
hydrogen_export_prices = hydrogen_prices["H2 export bus"]
hydrogen_export_prices.plot(
    title="Hourly Hydrogen Export Prices",
    xlabel="Time",
    ylabel="Units: (€ / MWh)",
    figsize=(20, 6)
)

In [ ]:
# Filter hydrogen export links and their quantities
hydrogen_export_links = n.links[n.links["carrier"] == "H2"]
hydrogen_export_quantity = n.links_t.p1[hydrogen_export_links.index]
hydrogen_export_quantity_cp = hydrogen_export_quantity.copy()
# Identify columns with export data dynamically (e.g., based on a naming pattern)
export_columns = [
    col for col in hydrogen_export_quantity_cp.columns if "H2 export" in col
]

# Sum the identified export columns into a new column named 'H2 export'
hydrogen_export_quantity_cp["H2 export"] = hydrogen_export_quantity_cp[export_columns].sum(axis=1)

# Drop the original separate export columns
hydrogen_export_quantity_cp = hydrogen_export_quantity_cp.drop(export_columns, axis=1)

# Plot the results
hydrogen_export_quantity_cp.plot(
    title="Hourly Hydrogen Export Quantity",
    xlabel="Time",
    ylabel="Units: (MWh)",
    figsize=(20, 6)
)

In [ ]:
# Revenue
hydrogen_export_quantity.set_index(hydrogen_export_prices.index, inplace=True)
hydrogen_export_quantity_cp["Hourly Revenue"] = (
    hydrogen_export_quantity_cp["H2 export"] * hydrogen_export_prices
)

hydrogen_export_quantity_cp["Hourly Revenue"].plot(
    title="Hourly Hydrogen Export Revenue",
    xlabel="Time",
    ylabel="Units: (€)",
    figsize=(20, 6)
)

#### H2 Export on a yearly basis

In [ ]:
H2_export_TWh = n.statistics.energy_balance(comps=["Load"]).loc[:,"H2"].values[0]/1e6 # in TWh
print(f"Yearly Hydrogen Export Quantity is: {H2_export_TWh} TWh")
H2_export_revenue = H2_export_TWh*h2_market
print(f"Hydrogen Export Revenue (yearly) is: {H2_export_revenue.round(2)} Mio. €")

### Hydrogen Capacity

In [ ]:
installed_capacity = n.statistics.installed_capacity(comps='Link').loc["H2 Electrolysis"]/ 1e3
expanded_capacity = n.statistics.expanded_capacity(comps='Link').loc["H2 Electrolysis"]/ 1e3
optimal_capacity = n.statistics.optimal_capacity(comps='Link').loc["H2 Electrolysis"]/ 1e3

categories = ['Installed Capacity', 'Expanded Capacity', 'Optimal Capacity']
values = [installed_capacity, expanded_capacity, optimal_capacity]

plt.figure()
plt.bar(categories, values)
plt.title('Comparison of H2 Electrolysis Capacities')
plt.ylabel('Capacity (GW)') 
plt.xlabel('Capacity Types')
plt.tight_layout()
plt.show()